In [1]:
import pandas as pd
import wrds

In [4]:
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [5]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.dsp500list as a,
                        crsp.dsf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2023'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [6]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]

In [10]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]

In [34]:
sp500_full

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
7,19393,1957-03-01,2023-12-29,2023-01-03,0.004309,BRISTOL MYERS SQUIBB CO,11012210,2004-06-10,2023-12-29,11,1,2834,BMY
11,12558,2012-05-25,2023-12-29,2023-01-03,-0.003872,KINDER MORGAN INC,49456B10,2020-11-03,2023-12-29,11,1,4924,KMI
24,86111,2007-11-19,2023-12-29,2023-01-03,0.013264,AMERICAN TOWER CORP NEW,03027X10,2021-04-05,2023-12-29,18,1,6798,AMT
37,85926,1998-04-01,2023-12-15,2023-01-03,0.007819,SEALED AIR CORP NEW,81211K10,2021-04-01,2023-12-29,11,1,2673,SEE
47,44644,1981-02-26,2023-12-29,2023-01-03,-0.005024,AUTOMATIC DATA PROCESSING INC,05301510,2020-05-04,2023-12-29,11,3,7374,ADP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886765,34032,1981-06-18,2023-12-29,2023-12-29,-0.002829,MASCO CORP,57459910,2021-04-01,2023-12-29,11,1,3432,MAS
886771,91233,2008-07-18,2023-12-29,2023-12-29,0.000446,MASTERCARD INC,57636Q10,2021-04-01,2023-12-29,11,1,7389,MA
886781,15720,1925-12-31,2023-12-29,2023-12-29,-0.001536,EDISON INTERNATIONAL,28102010,2014-02-26,2023-12-29,11,1,4911,EIX
886788,89195,2002-07-22,2023-12-29,2023-12-29,-0.007319,PRINCIPAL FINANCIAL GROUP INC,74251V10,2017-12-18,2023-12-29,11,3,6321,PFG


In [8]:
sp500_full[sp500_full['ticker']=='A']

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
1324,87432,2000-06-05,2023-12-29,2023-01-03,0.002606,AGILENT TECHNOLOGIES INC,00846U10,2021-04-01,2023-03-07,11,1,3826,A
5655,87432,2000-06-05,2023-12-29,2023-01-04,0.010864,AGILENT TECHNOLOGIES INC,00846U10,2021-04-01,2023-03-07,11,1,3826,A
9829,87432,2000-06-05,2023-12-29,2023-01-05,0.002901,AGILENT TECHNOLOGIES INC,00846U10,2021-04-01,2023-03-07,11,1,3826,A
12952,87432,2000-06-05,2023-12-29,2023-01-06,-0.029189,AGILENT TECHNOLOGIES INC,00846U10,2021-04-01,2023-03-07,11,1,3826,A
17467,87432,2000-06-05,2023-12-29,2023-01-09,-0.001354,AGILENT TECHNOLOGIES INC,00846U10,2021-04-01,2023-03-07,11,1,3826,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
869497,87432,2000-06-05,2023-12-29,2023-12-22,0.004534,AGILENT TECHNOLOGIES INC,00846U10,2023-03-08,2023-12-29,11,1,3826,A
874143,87432,2000-06-05,2023-12-29,2023-12-26,0.001720,AGILENT TECHNOLOGIES INC,00846U10,2023-03-08,2023-12-29,11,1,3826,A
876653,87432,2000-06-05,2023-12-29,2023-12-27,0.000072,AGILENT TECHNOLOGIES INC,00846U10,2023-03-08,2023-12-29,11,1,3826,A
881053,87432,2000-06-05,2023-12-29,2023-12-28,-0.000358,AGILENT TECHNOLOGIES INC,00846U10,2023-03-08,2023-12-29,11,1,3826,A
